# EK381 Exploratory Data Analysis <br> Lab 5: Pairs of Random Variables, Scatter Plots, and Monte Carlo Simulation

### This lab was completed by Rawisara Chairat.

This lab provides a few options to visualize pairs of random variables, with a focus on the continuous case. As we've seen in class, we can answer probability questions for pairs of random variables using multivariable integration. However, this does not provide us with much intuition. Here, we will start by plotting the 3D surface plot for the joint PDF for a few examples. Next, we will try generating pairs of random variables from these joint PDFs and plotting the resulting values along with the joint PDF. This will lead us to only plotting the data top down with what is known as a scatter plot. Finally, we will use a collection of scatter plots to uncover relationships between variables in the [breast cancer cell dataset](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic). 

**Overall, the goal of this lab is to get you comfortable working with 2-d scatter plots, since we will use these extensively for the remaining labs. You should make sure to take the time to understand the concept of a scatter plot, even if you are able to complete the lab quickly.**

**Also, when you are ready to turn in the lab, please be sure to clear the output of all cells. Gradescope has trouble with larger Jupyter notebooks, and this one will be quite large if all the images are preserved. There is a graded problem at the end that ony asks you to clear cell outputs, with instructions.**

In [ ]:
#numpy is a package for working with arrays, linear algebra, etc. and stands for "numerical Python".
import numpy as np
#pandas is a package for working with data frames.
import pandas as pd
#matplotlib is a plotting library that mimics Matlab conventions.
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#seaborn is a data visualization package.
import seaborn as sns
#scipy.stats is a statistics library
import scipy.stats as st

For this lab, we have provided the *rejection sampling* function below that can generate pairs of random variables from joint PDFs specified using equations very similar to those we use in EK381. Specifically, it takes in a joint PDF of the form
$$f_{X,Y}(x,y) = \begin{cases} g(x,y) & (x,y) \in R_{X,Y} \\ 0 & \text{otherwise}\end{cases}$$ where $R_{X,Y}$ is the range and $g(x,y)$ is a function that describes the joint PDF only within the range. The idea behind rejection sampling, simply put, is to initially draw $(X,Y)$ according to uniform distribution, and then only accept the sample $(X,Y)$ if a Uniform(0,1) random variable is less than the value of $g(X,Y)$. Optional (will not be tested in any way in EK381): Here is a [2-minute video visualizing rejection sampling](https://www.youtube.com/watch?v=7MdJstRhveg) (for generating $X \sim f_X(x)$ given access only to Uniform(0,1) randomness) as well as the [Wikipedia page.](https://en.wikipedia.org/wiki/Rejection_sampling) This is a kind of [Monte Carlo simulation](https://en.wikipedia.org/wiki/Monte_Carlo_method) that uses (synthetic) randomness to solve problems in engineering and beyond.

In [ ]:
#This function generates number_samples of the pair of random variables (X,Y) using rejection sampling. 
#We will not cover the concept of rejection sampling in EK381, but this function will allow us to generate
#samples from any joint distribution, which will help with our visualizations.
def f_XY_generate_samples(g, R_XY, plotrangeX, plotrangeY, number_samples):
    #First, we need to find the maximum value of g in the plotting range.
    x_grid = np.linspace(*plotrangeX,100)
    y_grid = np.linspace(*plotrangeY,100)
    x_mesh, y_mesh = np.meshgrid(x_grid,y_grid)
    max_g = np.max(g(x_mesh,y_mesh))
    XYsamples = []
    while len(XYsamples) < number_samples:
        # Generate random X and Y values over a box containing the range.
        X = np.random.uniform(*plotrangeX)
        Y = np.random.uniform(*plotrangeY)
        # Check if the candidate sample is inside the range
        if rangeXY(X,Y):
            # Only accept the sample (X,Y) if a Uniform(0,1) RV is less than g(X,Y)
            prob = np.random.uniform(0, 1)
            if prob < (g(X,Y)/np.max([max_g, 1])):
                XYsamples.append((X,Y))
    # The accepted samples will actually have joint PDF f_XY even though the candidate samples were uniform.
    return np.array(XYsamples)

Let's try this out on a few examples. First, consider the joint PDF
$$f_{XY}(x,y) = \begin{cases} \displaystyle \frac{1}{2} & 0 \leq x \leq 2, 0 \leq y \leq 1 \\ 0 & \text{otherwise} \end{cases}$$
Below, we've called `f_XY_generate_samples` to obtain 10 samples from this distribution. (The parameters `plotrangeX` and `plotrangeY` need to be chosen so that the resulting rectangle in the x-y plane contains the range $R_{XY}$, referred to as `rangeXY` in the function. We've handled this for you throughout the lab.)

In [ ]:
#Define the density function that is active over the range.
def g(X, Y):
    return 0.5

# Define the range.
def rangeXY(X, Y):
    return (0 <= X) & (X <= 2) & (0 <= Y) & (Y <= 1)  
    
# Generate 10 samples from the distribution and print them out.
# Notice that we are setting plotrangeX so that 0 <= X <= 2 and 0 <= Y <= 1
XYsamples = f_XY_generate_samples(g, rangeXY, plotrangeX=(0,2), plotrangeY=(0,1), number_samples=1000)
samples_df = pd.DataFrame(columns=["X","Y"],data=XYsamples)
samples_df

Now, let's use a scatter plot to look at these points on the x-y plane. There are a few ways to do this. Below, we are using `matplotlib`, but later we will use `seaborn`. 

In [ ]:
plt.scatter(x = samples_df["X"], y= samples_df["Y"])
plt.xlim(0,2)
plt.xlabel("X")
plt.ylim(0,1)
plt.ylabel("Y")

---
####  L5.1 Try increasing the number of sample points and regenerating the scatter plot. For the 10 points we initially generated, it is hard to get a sense for the distribution. Roughly how many samples do you need before you can visually see that they are uniformly generated over the rectangle?
---
#### Answer: Around 1,000 samples are where it looks uniformly generated over the rectangle.

Now, let's take this a bit further and simultaneously visualize the 3d surface plot corresponding to the joint PDF along with 2d scatter plot corresponding to the (X,Y) samples. The function below does this for you (with `grid_size` determining the plotting resolution for the 3d surface plot).

In [ ]:
def plot_XY_samples_and_joint_pdf(g, rangeXY, plotrangeX, plotrangeY, number_samples, grid_size=200):
    # Create a grid of (x,y) points
    x = np.linspace(*plotrangeX,grid_size)
    y = np.linspace(*plotrangeY,grid_size)
    X, Y = np.meshgrid(x,y)

    # Compute g(x,y) over the grid, setting values outside rangeXY to 0
    f_XY = np.zeros_like(X)
    for i in range(grid_size):
        for j in range(grid_size):
            if rangeXY(X[i,j], Y[i,j]):
                f_XY[i,j] = g(X[i,j], Y[i,j])

    # Create the figure and the 3d axis
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the joint PDF as 3d surface plot
    ax.plot_surface(X, Y, f_XY, cmap='Blues', edgecolor='none', alpha=0.5)

    # Add the samples as a scatter plot on the 2d x-y plane
    XYsamples = f_XY_generate_samples(g, rangeXY, plotrangeX, plotrangeY, number_samples)
    ax.scatter(XYsamples[:, 0], XYsamples[:, 1], np.zeros(len(XYsamples)), label='Samples')
        
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$f_{X,Y}(x,y)$')
    ax.zaxis.labelpad= 0.7
    ax.set_title('Joint PDF and Samples')
    plt.ion()
    ax.view_init(elev=45, azim=-60)  
    ax.legend()
    plt.tight_layout()

    return np.array(XYsamples)

In [ ]:
# g(x,y) encodes a constant height of 1/2 inside the range
def g(x,y):
    return 1/2

# rangeXY encodes 0 <= x <= 2, 0 <= y <= 1
def rangeXY(x, y):
    return (0 <= x) & (x <= 2) & (0 <= y) & (y <= 1)  

rectangle_samples = plot_XY_samples_and_joint_pdf(g, rangeXY, plotrangeX=(-2, 2), plotrangeY=(-2, 2), number_samples = 200)

You should see a box over the range with height $\frac{1}{2}$. This represents the joint PDF. You should also see 200 samples beneath the box. Let's try this with a different joint PDF that does not have a rectangular range. 

In [ ]:
# g(x,y) encodes the density within the range, which is of constant height
def g(x,y):
    return 2

# rangeXY encodes a triangle with vertices (-1,0), (0,1), (1,0)  
def rangeXY(x, y):
    return (x + y <= 1) & (-x + y <= 1) & (0 <= y)  

triangle_samples = plot_XY_samples_and_joint_pdf(g, rangeXY, plotrangeX=(-2, 2), plotrangeY=(-1, 2), number_samples = 200)
triangle_df = pd.DataFrame(columns=["X","Y"],data=triangle_samples)

You should see an elevated triangle of height $2$ representing the joint PDF along with the generated data in the triangular footprint underneath. Let's now use `seaborn` to quickly generate a scatter plot. 

In [ ]:
sns.scatterplot(data=triangle_df,x="X",y="Y")

Now, let's look at a joint PDF whose density varies. This is the joint PDF corresponding to the case where $X$ and $Y$ are independent Gaussian$(0,1)$.

In [ ]:
# g(x,y) encodes the density within the range, which varies with respect to x and y.
def g(x,y):
    return 1/(2*np.pi) * np.exp(-(x**2 + y**2))

# rangeXY encodes all x,y
def rangeXY(x, y):
    return 1 

gaussian_samples = plot_XY_samples_and_joint_pdf(g, rangeXY, plotrangeX=(-3,3), plotrangeY=(-3,3), number_samples = 1000)
gaussian_df = pd.DataFrame(columns=["X","Y"],data=gaussian_samples)

It turns out there is a convenient `seaborn` command `sns.jointplot` to simultaneously display the scatter plot as well as the histograms corresponding to the $X$ data and the $Y$ data, individually.

In [ ]:
sns.jointplot(data=gaussian_df,x="X",y="Y")

Now, let's look at a joint PDF from the homework.

In [ ]:
# g(x,y) encodes the density within the range, which varies with respect to x and y.
def g(x,y):
    return 3/4 * x**2 * y

# rangeXY encodes -1 <= x <= 1, 0 <= y <= 2
def rangeXY(x, y):
    return (-1 <= x) & (x <= 1) & (0 <= y) & (y <= 2)  

HW_samples = plot_XY_samples_and_joint_pdf(g, rangeXY, plotrangeX=(-1, 1), plotrangeY=(0, 2), number_samples = 2000)
HW_df = pd.DataFrame(columns=["X","Y"],data=HW_samples)

In the homework, you were asked to work out various probabilities and expectations for this joint PDF using multi-variable integration. It turns out that, using Monte Carlo simulation, we can estimate these quantities. Let's work out an example.

Let's say we want to estimate $\mathbb{P}[X \leq 0, Y \leq 1]$. We know from the homework that this corresponds to the double integral 
$$\mathbb{P}[X \leq 0, Y \leq 1] = \int_{-1}^0 \int_0^1 \frac{3}{4} x^2\,  y\,  dy \, dx = \frac{1}{8} = 0.125$$ 

To estimate this, we just need to determine how many times the condition is satisfied in the dataset, and divide that by the total number of samples. An easy way to do this is to use the `value_counts(normalize=True)` command.

In [ ]:
((HW_df["X"] <= 0) & (HW_df["Y"] <= 1)).value_counts(normalize=True)

---
####  L5.2 In the cell below, estimate the probability that $Y$ is less than $X$ from the data and type your estimated value (with 3 digital after the decimal point) below. 
---

#### Answer: 0.071

In [ ]:
(HW_df["Y"] < HW_df["X"]).value_counts(normalize=True)

We can also use Monte Carlo simulation to estimate expectations. We have already seen this for estimating the mean $\mathbb{E}[X]$, but what about functions of a pair of random variables?  Below, we've done this for $\mathbb{E}[X^4 Y]$, which appears in the homework. Recall that `**` is used to mean "to the power of" in Python. The estimated value should be close to the true value obtained from the integral
$$\mathbb{E}[X^4 Y] = \int_{-1}^1 \int_0^2 \frac{3}{4} x^4 y \, dy \, dx = \frac{4}{7} = 0.571\ldots$$

In [ ]:
(HW_df["X"]**4 * HW_df["Y"]).mean()

---
####  L5.3 In the cell below, estimate the expectation $\mathbb{E}[X^3 - 2Y^2]$ and type your estimated value (with 3 digits after the decimal point) below. 
---

#### Answer: -4.025

In [ ]:
(HW_df["X"]**3 - 2*HW_df["Y"]**2).mean()

In [ ]:
XYsamples = f_XY_generate_samples(g, rangeXY, plotrangeX=(-1,1), plotrangeY=(0,2), number_samples=2000)
samples_df = pd.DataFrame(columns=["X","Y"],data=XYsamples)
sns.scatterplot(data=samples_df,x = "X", y = "Y")

Now, let's load a real dataset and use the scatter plot to discover relationships in the dataset. (This dataset description appeared in Lab 4 so can skip this if you remember it: The [Diagnostic Wisconsin Breast Cancer Database](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) is a collection of measurements based on the images of cells collected from benign and malignant tissues as part of the paper W. Street, W. Wolberg, O. Mangasarian, "Nuclear feature extraction for breast tumor diagnosis," Electronic Imaging, 1993. ([PDF Link](https://minds.wisconsin.edu/bitstream/1793/59692/1/TR1131.pdf)) Once you load the dataset below, you will see there are 12 columns: an ID column, a diagnosis column that contains "B" for benign and "M" for malignant, and 10 additional columns corresponding to different features (e.g., Texture, Perimeter). The goal of the dataset is to study how well these features can be used to classify cells as benign or malignant, and we will consider that question directly in a subsequent lab.

In [ ]:
bc_df = pd.read_csv("datasets/breastcancercells.csv")
bc_df

Let's drop the first 3 columns, since we will not need the index or ID number in our analysis, and it will make the syntax simpler.

In [ ]:
bc_noID_df = bc_df.drop(bc_df.columns[:2],axis=1)
bc_noID_df

Now, let's display scatter plots for all possible pairs of columns. The command `sns.pairplot` does this for us and organizes the results into a square. On the diagonal, we are plotting a variable against itself, which would not make a very interesting scatter plot, so it shows the variable's histogram instead. This visualization allows us to quickly identify interesting relationships in the data.

In [ ]:
sns.pairplot(bc_noID_df)

---
####  L5.4 Can you identify two columns that have a linear relationship (i.e., the scatter plot is essentially a line)? Explain why it makes sense that their relationships is linear based on the column names.
---

#### Answer: Radius and Perimeter. It is a linear relationship since perimeter of the circle is linearly related to its radius by the formula: $$Perimeter = 2πR$$

---
####  L5.5 Can you identify two columns that have a quadratic relationship (i.e., the data in one column can be interpreted as the square of the data in the other column, perhaps scaled by a constant)? Explain why it makes sense that their relationships is quadratic based on the column names.
---

#### Answer: Area and Radius. It is a quadratic relationship since area is propotional to the square of the radius as in this formula:  $$ Area = πR^2$$

Finally, let's look at how we can use `sns.pairplot` with the `hue` parameter to visualize how well we can do classification based on a pair of values. In Lab 4, we noticed that, using the "Radius" information only, we can do a reasonable job of classifying cells as "Benign" or "Malignant". The code below regenerates the grid of scatter plots, but now uses blue for "Malignant" and orange for "Benign."

In [ ]:
sns.pairplot(bc_noID_df,hue="Diagnosis")

---
####  L5.6 Write down two column names for which the "Malignant" and "Benign" data are well separated. (There are many right answers, just pick one pair of names.)
---

#### Answer: Perimeter and Symmetry

---
####  L5.7 Write down two column names for which the "Malignant" and "Benign" data are poorly separated. (There are many right answers, just pick one pair of names.)
---

#### Answer: Fractal Dimension and Symmetry

---
####  L5.8 Finally, to make our grading process easier, please make sure your answers are written in the "Answer:" fields above and then clear the outputs of all cells before uploading your lab for grading. 
---
Instructions: 
In Jupyter Notebook or Jupyter Lab, use the "Edit" menu and then "Clear Outputs of All Cells".
In Google Colab, use the "Edit" menu and then "Clear all outputs".

